In [27]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 64
batch_size = 28
max_iters=3000
learning_rate = 3e-4
eval_iters = 100
n_embd = 384
n_layer= 8
n_head= 8
dropout = 0.2

cuda


In [28]:
with open('Script.txt', 'r', encoding='utf-8') as f:
          text =f.read()
print(len(text))

3357505


In [29]:
chars= sorted(set(text))
print(chars)
vocab_size = len(chars)
vocab_size

['\t', '\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ';', '<', '=', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '}', '~', '\x91', '\x92', '\x93', '\x94']


97

In [30]:
string_to_int = {ch:i for i, ch in enumerate(chars) }
int_to_string = {i:ch for i, ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''. join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([35, 71, 64, 81, 64, 66, 83, 68, 81,  0, 36, 72, 64, 75, 78, 70, 84, 68,
         1, 42, 37, 50, 50, 57,  0,  4, 36, 78,  2, 88, 78, 84,  2, 74, 77, 78,
        86,  2, 86, 71, 64, 83,  2, 83, 71, 72, 82,  2, 72, 82,  2, 64, 75, 75,
         2, 64, 65, 78, 84, 83, 32,  2, 36, 78,  2, 88, 78, 84,  2, 74, 77, 78,
        86, 14,  2, 86, 71, 88,  2, 86, 68, 81, 68,  2, 71, 68, 81, 68, 32,  2,
        52, 78,  2, 65, 68,  2, 78, 84, 83, 14])


In [31]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data=train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x, y
    
x, y = get_batch('train')
print('inputs:')
print(x)
print('targets: ')
print(y)

inputs:
tensor([[83, 71, 72,  ..., 72, 77, 70],
        [78, 83,  2,  ..., 41, 46, 37],
        [81, 83, 84,  ..., 78, 82, 82],
        ...,
        [69, 69, 68,  ..., 82, 88, 81],
        [71, 68,  2,  ...,  1, 39, 37],
        [68, 67,  2,  ...,  2, 66, 64]], device='cuda:0')
targets: 
tensor([[71, 72, 77,  ..., 77, 70,  2],
        [83,  2, 66,  ..., 46, 37,  0],
        [83, 84, 77,  ..., 82, 82, 14],
        ...,
        [69, 68, 68,  ..., 88, 81, 84],
        [68,  2, 72,  ..., 39, 37, 47],
        [67,  2, 78,  ..., 66, 64, 82]], device='cuda:0')


In [32]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [33]:

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

# [1, 0, 0]
# [1, 0.6, 0]
# [1, 0.6, 0.4]
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # (B, T, F) -> (B, T, [h1, h1, h1, h1, h2, h2, h2, h2, h3, h3, h3, h3])
        out = self.dropout(self.proj(out))
        return out
    

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)
    
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x
    
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
        
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
        B, T = index.shape
        
        
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(index) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            index_cond = index[:, -block_size:]
            # get the predictions
            logits, loss = self.forward(index_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = GPTLanguageModel(vocab_size)
# print('loading model parameters...')
# with open('model-01.pkl', 'rb') as f:
#     model = pickle.load(f)
# print('loaded successfully!')
m = model.to(device)

In [34]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    if iter % eval_iters==0:
        losses=estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']}, val loss: {losses['val']}")
    xb, yb = get_batch('train')

    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.654092788696289, val loss: 4.653753757476807
step: 100, train loss: 2.3546597957611084, val loss: 2.3944263458251953
step: 200, train loss: 2.073500871658325, val loss: 2.1192686557769775
step: 300, train loss: 1.917709469795227, val loss: 1.9626259803771973
step: 400, train loss: 1.798179268836975, val loss: 1.8418371677398682
step: 500, train loss: 1.7343897819519043, val loss: 1.7819987535476685
step: 600, train loss: 1.6777398586273193, val loss: 1.725405216217041
step: 700, train loss: 1.6193426847457886, val loss: 1.680756688117981
step: 800, train loss: 1.590999960899353, val loss: 1.6544233560562134
step: 900, train loss: 1.5621061325073242, val loss: 1.6429836750030518
step: 1000, train loss: 1.5340019464492798, val loss: 1.6071815490722656
step: 1100, train loss: 1.518387794494629, val loss: 1.5982332229614258
step: 1200, train loss: 1.5047712326049805, val loss: 1.5847097635269165
step: 1300, train loss: 1.480350136756897, val loss: 1.5604884624481201


In [39]:
%%capture cap
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

	(looks oce uglichent) I'm very a hampolway dass an luckle.
JERRY	Huh, if you dont know per go sliftel?
GEORGE	I can just look me thing do show Golda stanzed you what she would go head me that? I can't, it be clarl to do the hell tob, I can do it. That's hight, that, readyer."
JERRY & KRAMER	Get!
KRAMER	Yeah?
JERRY	"Canter to busy. Elaine, Like, I said I talk...."
KRAMER	"Okay."
ELAINE	"Ah, if if that's Jerry wenty avoing an here."
MONGICE	get an is for selasy at Knight?
JERRY	"Hey , ity goves."

